# CenterTrack-Stable

In [ ]:
def reward_function(params):

    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    abs_steering = abs(params['steering_angle']) # Only need the absolute steering angle

    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.1 * track_width
    marker_2 = 0.25 * track_width
    marker_3 = 0.5 * track_width

    # Give higher reward if the car is closer to center line and vice versa
    if distance_from_center <= marker_1:
        reward = 1.0
    elif distance_from_center <= marker_2:
        reward = 0.5
    elif distance_from_center <= marker_3:
        reward = 0.1
    else:
        reward = 1e-3  # likely crashed/ close to off track

    # Steering penality threshold, change the number based on your action space setting
    ABS_STEERING_THRESHOLD = 15

    # Penalize reward if the car is steering too much
    if abs_steering > ABS_STEERING_THRESHOLD:
        reward *= 0.8
    return float(reward)

# EdgeAware-SpeedOptimized

In [ ]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward + 2 * progress_reward) / 7

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.3  # to allow going outside the track by sense :)

    return float(reward)


# TrackAware-Optimized

In [ ]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']
    off_track = params['is_offtrack']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward + 2 * progress_reward) / 7

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.5  # to allow going outside the track by sense :)

    # Penalize if the car goes off track
    if off_track:
        reward *= 0.5

    return float(reward)


# AdaptivePathOptimizer

In [ ]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']
    off_track = params['is_offtrack']

    # Waypoints data
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    heading = params['heading']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Track direction alignment using waypoints
    wp1 = waypoints[closest_waypoints[0]]
    wp2 = waypoints[closest_waypoints[1]]

    track_direction = math.atan2(wp2[1] - wp1[1], wp2[0] - wp1[0]) * 180.0 / math.pi
    direction_diff = abs(track_direction - heading)

    # Normalize direction reward (similar to other rewards)
    direction_ratio = min(direction_diff / 30, 1)  # Normalize to 0-1 range
    direction_reward = 1 - math.sqrt(direction_ratio)  # Similar transformation

    # Adjust speed based on upcoming waypoints
    if direction_diff < 10:
        speed_bonus = speed / 4.0  # Encourage speed on straight paths
    else:
        speed_bonus = (4.0 - speed) / 4.0  # Encourage slowing down on turns

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward +
              2 * progress_reward + 2 * direction_reward + speed_bonus) / 9

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.5  # Allow minor track deviation

    if off_track:
        reward *= 0.5  # Strong penalty for completely going off track

    return float(reward)


# AdaptivePathOptimizer-Speed

In [ ]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']
    off_track = params['is_offtrack']

    # Waypoints data
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    heading = params['heading']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Track direction alignment using waypoints
    wp1 = waypoints[closest_waypoints[0]]
    wp2 = waypoints[closest_waypoints[1]]

    track_direction = math.atan2(wp2[1] - wp1[1], wp2[0] - wp1[0]) * 180.0 / math.pi
    direction_diff = abs(track_direction - heading)

    # Normalize direction reward (similar to other rewards)
    direction_ratio = min(direction_diff / 30, 1)  # Normalize to 0-1 range
    direction_reward = 1 - math.sqrt(direction_ratio)  # Similar transformation

    # Adjust speed based on upcoming waypoints
    if direction_diff < 10:
        speed_bonus = speed / 4.0  # Encourage speed on straight paths
    else:
        speed_bonus = (4.0 - speed) / 4.0  # Encourage slowing down on turns

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward +
              2 * progress_reward + 2 * direction_reward + speed_bonus) / 10

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.5  # Allow minor track deviation

    if off_track:
        reward *= 0.5  # Strong penalty for completely going off track

    return float(reward)


# AdaptivePathOptimizer-Speed-V2

In [ ]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']
    off_track = params['is_offtrack']

    # Waypoints data
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    heading = params['heading']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Track direction alignment using waypoints
    wp1 = waypoints[closest_waypoints[0]]
    wp2 = waypoints[closest_waypoints[1]]

    track_direction = math.atan2(wp2[1] - wp1[1], wp2[0] - wp1[0]) * 180.0 / math.pi
    direction_diff = abs(track_direction - heading)

    # Normalize direction reward (similar to other rewards)
    direction_ratio = min(direction_diff / 30, 1)  # Normalize to 0-1 range
    direction_reward = 1 - math.sqrt(direction_ratio)  # Similar transformation

    # Adjust speed based on upcoming waypoints
    if direction_diff < 10:
        speed_bonus = speed / 4.0  # Encourage speed on straight paths
    else:
        speed_bonus = (4.0 - speed) / 4.0  # Encourage slowing down on turns

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward +
              2 * progress_reward + 2 * direction_reward + speed_bonus) / 10

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.5  # Allow minor track deviation

    if off_track:
        reward *= 0.5  # Strong penalty for completely going off track

    return float(reward)


# AdaptivePathOptimizer-Speed-V3

In [ ]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']
    off_track = params['is_offtrack']

    # Waypoints data
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    heading = params['heading']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Track direction alignment using waypoints
    wp1 = waypoints[closest_waypoints[0]]
    wp2 = waypoints[closest_waypoints[1]]

    track_direction = math.atan2(wp2[1] - wp1[1], wp2[0] - wp1[0]) * 180.0 / math.pi
    direction_diff = abs(track_direction - heading)

    # Normalize direction reward (similar to other rewards)
    direction_ratio = min(direction_diff / 30, 1)  # Normalize to 0-1 range
    direction_reward = 1 - math.sqrt(direction_ratio)  # Similar transformation

    # Adjust speed based on upcoming waypoints
    if direction_diff < 10:
        speed_bonus = speed / 4.0  # Encourage speed on straight paths
    else:
        speed_bonus = (4.0 - speed) / 4.0  # Encourage slowing down on turns

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward +
              2 * progress_reward + 2 * direction_reward + speed_bonus) / 10

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.5  # Allow minor track deviation

    if off_track:
        reward *= 0.5  # Strong penalty for completely going off track

    return float(reward)


# AdaptivePathOptimizer-Left-Speed

In [ ]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']
    off_track = params['is_offtrack']

    # Waypoints data
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    heading = params['heading']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Track direction alignment using waypoints
    wp1 = waypoints[closest_waypoints[0]]
    wp2 = waypoints[closest_waypoints[1]]

    track_direction = math.atan2(wp2[1] - wp1[1], wp2[0] - wp1[0]) * 180.0 / math.pi
    direction_diff = abs(track_direction - heading)

    # Normalize direction reward (similar to other rewards)
    direction_ratio = min(direction_diff / 30, 1)  # Normalize to 0-1 range
    direction_reward = 1 - math.sqrt(direction_ratio)  # Similar transformation

    # Adjust speed based on upcoming waypoints
    if direction_diff < 10:
        speed_bonus = speed / 4.0  # Encourage speed on straight paths
    else:
        speed_bonus = (4.0 - speed) / 4.0  # Encourage slowing down on turns

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward +
              2 * progress_reward + 2 * direction_reward + speed_bonus) / 10

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.5  # Allow minor track deviation

    if off_track:
        reward *= 0.5  # Strong penalty for completely going off track

    return float(reward)


# AdaptivePathOptimizer-High-Entropy

In [1]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']
    off_track = params['is_offtrack']

    # Waypoints data
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    heading = params['heading']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Track direction alignment using waypoints
    wp1 = waypoints[closest_waypoints[0]]
    wp2 = waypoints[closest_waypoints[1]]

    track_direction = math.atan2(wp2[1] - wp1[1], wp2[0] - wp1[0]) * 180.0 / math.pi
    direction_diff = abs(track_direction - heading)

    # Normalize direction reward (similar to other rewards)
    direction_ratio = min(direction_diff / 30, 1)  # Normalize to 0-1 range
    direction_reward = 1 - math.sqrt(direction_ratio)  # Similar transformation

    # Adjust speed based on upcoming waypoints
    if direction_diff < 10:
        speed_bonus = speed / 4.0  # Encourage speed on straight paths
    else:
        speed_bonus = (4.0 - speed) / 4.0  # Encourage slowing down on turns

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward +
              2 * progress_reward + 2 * direction_reward + speed_bonus) / 10

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.5  # Allow minor track deviation

    if off_track:
        reward *= 0.5  # Strong penalty for completely going off track

    return float(reward)


# AdaptivePathOptimizer-Speed-with-High-Entropy

In [ ]:
import math

def reward_function(params):
    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    is_left_of_center = params['is_left_of_center']
    abs_steering = abs(params['steering_angle'])
    speed = params['speed']
    progress = params['progress']
    all_wheels_on_track = params['all_wheels_on_track']
    off_track = params['is_offtrack']

    # Waypoints data
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    heading = params['heading']

    # Calculate distance from the left edge
    if is_left_of_center:
        distance_from_left_side = (track_width / 2) - distance_from_center
    else:
        distance_from_left_side = (track_width / 2) + distance_from_center

    # Ensure non-negative values before applying sqrt
    distance_ratio = max(0, distance_from_left_side / track_width)
    distance_reward = 1 - math.sqrt(distance_ratio)

    steering_ratio = max(0, abs_steering / 30)
    steering_reward = 1 - math.sqrt(steering_ratio)

    speed_reward = (speed / 4.0) ** 2
    progress_reward = progress / 100

    # Track direction alignment using waypoints
    wp1 = waypoints[closest_waypoints[0]]
    wp2 = waypoints[closest_waypoints[1]]

    track_direction = math.atan2(wp2[1] - wp1[1], wp2[0] - wp1[0]) * 180.0 / math.pi
    direction_diff = abs(track_direction - heading)

    # Normalize direction reward (similar to other rewards)
    direction_ratio = min(direction_diff / 30, 1)  # Normalize to 0-1 range
    direction_reward = 1 - math.sqrt(direction_ratio)  # Similar transformation

    # Adjust speed based on upcoming waypoints
    if direction_diff < 10:
        speed_bonus = speed / 4.0  # Encourage speed on straight paths
    else:
        speed_bonus = (4.0 - speed) / 4.0  # Encourage slowing down on turns

    # Combine rewards with weights
    reward = (2 * distance_reward + steering_reward + 2 * speed_reward +
              2 * progress_reward + 2 * direction_reward + speed_bonus) / 10

    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward *= 0.5  # Allow minor track deviation

    if off_track:
        reward *= 0.5  # Strong penalty for completely going off track

    return float(reward)
